# Digital Twin: Monitoring, Detection, and Control

The ReactorTwin digital twin layer adds a complete operational stack on top of Neural DE models:

- **EKF State Estimation** - Fuse model predictions with noisy measurements
- **Fault Detection** - Statistical Process Control (SPC) with EWMA and CUSUM charts, plus residual-based detection
- **MPC Control** - Model Predictive Control using the Neural ODE as a plant model
- **Online Adaptation** - Continual learning with Elastic Weight Consolidation (EWC)
- **Meta-Learning** - Fast adaptation to new operating conditions

This notebook demonstrates the full pipeline from training a plant model to deploying it as a digital twin with monitoring, fault detection, and optimal control.

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

from reactor_twin import (
    ArrheniusKinetics, CSTRReactor, NeuralODE,
    EKFStateEstimator, FaultDetector, MPCController, OnlineAdapter,
)
from reactor_twin.digital_twin import SPCChart

np.random.seed(42)
torch.manual_seed(42)

## Setup: Train a NeuralODE Plant Model

Before we can build a digital twin, we need a trained Neural ODE that captures the plant dynamics. We use a simple CSTR with an A -> B reaction as our reference system.

In [ ]:
kinetics = ArrheniusKinetics(
    name="A_to_B", num_reactions=1,
    params={"k0": np.array([0.5]), "Ea": np.array([0.0]),
            "stoich": np.array([[-1, 1]]), "orders": np.array([[1, 0]])},
)
reactor = CSTRReactor(
    name="cstr", num_species=2,
    params={"V": 10.0, "F": 1.0, "C_feed": [1.0, 0.0], "T_feed": 350.0,
            "C_initial": [0.8, 0.2]},
    kinetics=kinetics, isothermal=True,
)

state_dim = 2
y0 = reactor.get_initial_state()
dt = 0.1
t_eval = np.linspace(0, 10, 101)
sol = solve_ivp(reactor.ode_rhs, [0, 10], y0, t_eval=t_eval, method="LSODA")
true_states = sol.y.T

# Train NeuralODE
model = NeuralODE(state_dim=2, hidden_dim=32, num_layers=2, solver="rk4", adjoint=False)
z0_train = torch.tensor(y0, dtype=torch.float32).unsqueeze(0)
t_train = torch.tensor(t_eval[:51], dtype=torch.float32)
targets_train = torch.tensor(true_states[:51], dtype=torch.float32).unsqueeze(0)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    preds = model(z0_train, t_train)
    loss = model.compute_loss(preds, targets_train)["total"]
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()

print(f"Model trained. Final loss: {loss.item():.6f}")

## 1. EKF State Estimation

The Extended Kalman Filter (EKF) fuses the Neural ODE's predictions with noisy measurements to produce optimal state estimates. It uses the Neural ODE as the process model and linearizes it at each step for the Kalman update.

In [ ]:
# Generate noisy measurements
noise_std = 0.05
measurements_np = true_states[1:] + noise_std * np.random.randn(100, 2)
measurements_np = np.maximum(measurements_np, 0)
measurements = torch.tensor(measurements_np, dtype=torch.float32)

# Set up EKF
ekf = EKFStateEstimator(
    model=model, state_dim=2,
    Q=1e-3, R=noise_std**2, P0=0.1, dt=0.1,
)

z0_ekf = torch.tensor(y0, dtype=torch.float32)
t_ekf = torch.tensor(t_eval[1:], dtype=torch.float32)
result = ekf.filter(measurements=measurements, z0=z0_ekf, t_span=t_ekf)

filtered = result["states"].numpy()

# Compare
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
for i, name in enumerate(['C_A', 'C_B']):
    axes[i].plot(t_eval[1:], true_states[1:, i], 'k-', linewidth=2, label='True')
    axes[i].plot(t_eval[1:], measurements_np[:, i], '.', alpha=0.3, label='Measured')
    axes[i].plot(t_eval[1:], filtered[:, i], 'r-', linewidth=1.5, label='EKF Filtered')
    axes[i].set_xlabel('Time'); axes[i].set_ylabel(name)
    axes[i].set_title(f'EKF State Estimation: {name}')
    axes[i].legend(); axes[i].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

rmse_meas = np.sqrt(np.mean((measurements_np - true_states[1:])**2))
rmse_ekf = np.sqrt(np.mean((filtered - true_states[1:])**2))
print(f"RMSE (measurements): {rmse_meas:.4f}")
print(f"RMSE (EKF filtered): {rmse_ekf:.4f}")
print(f"Improvement: {rmse_meas/rmse_ekf:.1f}x")

## 2. Fault Detection with SPC Charts

Statistical Process Control (SPC) detects anomalies using two complementary methods:
- **EWMA** (Exponentially Weighted Moving Average) - Sensitive to small, sustained shifts
- **CUSUM** (Cumulative Sum) - Optimal for detecting mean shifts of known magnitude

We first establish a baseline from normal operation, then monitor for a step change fault.

In [ ]:
# Generate normal operation data
normal_data = np.column_stack([
    0.5 + 0.02 * np.random.randn(200),
    0.5 + 0.02 * np.random.randn(200),
])

# Set up SPC
spc = SPCChart(num_vars=2, ewma_lambda=0.2, ewma_L=3.0, cusum_k=0.5, cusum_h=5.0)
spc.set_baseline(normal_data)

# Test data with fault at sample 50
test_data = np.column_stack([
    0.5 + 0.02 * np.random.randn(100),
    0.5 + 0.02 * np.random.randn(100),
])
test_data[50:, 0] += 0.1  # Step change in C_A

# Monitor
ewma_alarms = []
cusum_alarms = []
for i in range(100):
    result = spc.update(test_data[i])
    ewma_alarms.append(result["ewma_alarm"].any())
    cusum_alarms.append(result["cusum_alarm"].any())

fig, axes = plt.subplots(2, 1, figsize=(12, 6))
axes[0].plot(test_data[:, 0], 'b-', label='C_A')
axes[0].axvline(x=50, color='r', linestyle='--', label='Fault onset')
axes[0].axhline(y=0.5, color='k', linestyle=':', alpha=0.5)
axes[0].set_ylabel('C_A'); axes[0].legend(); axes[0].grid(True, alpha=0.3)
axes[0].set_title('Fault Detection: Step Change in C_A')

alarm_timeline = np.array([1 if e else 0 for e in ewma_alarms])
cusum_timeline = np.array([1 if c else 0 for c in cusum_alarms])
axes[1].fill_between(range(100), alarm_timeline, alpha=0.3, label='EWMA Alarm', color='orange')
axes[1].fill_between(range(100), cusum_timeline * 0.5, alpha=0.3, label='CUSUM Alarm', color='red')
axes[1].axvline(x=50, color='r', linestyle='--')
axes[1].set_xlabel('Sample'); axes[1].set_ylabel('Alarm')
axes[1].legend(); axes[1].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

first_ewma = next((i for i, a in enumerate(ewma_alarms) if a), None)
first_cusum = next((i for i, a in enumerate(cusum_alarms) if a), None)
print(f"First EWMA alarm: sample {first_ewma} (delay: {first_ewma - 50 if first_ewma else 'N/A'})")
print(f"First CUSUM alarm: sample {first_cusum} (delay: {first_cusum - 50 if first_cusum else 'N/A'})")

## 3. Model Predictive Control (MPC)

MPC uses the Neural ODE as a plant model to compute optimal control actions over a receding horizon. At each step, it solves an optimization problem to find the control sequence that minimizes the tracking error while respecting constraints.

In [ ]:
# Create model with control input
model_ctrl = NeuralODE(
    state_dim=2, input_dim=1, hidden_dim=32, num_layers=2,
    solver="rk4", adjoint=False,
)

# Train with Euler rollout (MPC uses ode_func directly)
targets_ctrl = torch.tensor(sol.y.T, dtype=torch.float32)
dt_train = t_eval[1] - t_eval[0]
optimizer = torch.optim.Adam(model_ctrl.parameters(), lr=1e-3)
model_ctrl.train()
for epoch in range(200):
    optimizer.zero_grad()
    z = torch.tensor(y0, dtype=torch.float32).unsqueeze(0)
    u_zero = torch.zeros(1, 1)
    total_loss = torch.tensor(0.0)
    for k in range(20):
        dzdt = model_ctrl.ode_func(torch.tensor(0.0), z, u_zero)
        z = z + dzdt * dt_train
        total_loss = total_loss + torch.mean((z - targets_ctrl[k+1].unsqueeze(0))**2)
    (total_loss / 20).backward()
    torch.nn.utils.clip_grad_norm_(model_ctrl.parameters(), max_norm=1.0)
    optimizer.step()

# Set up MPC
mpc = MPCController(model=model_ctrl, horizon=5, dt=0.2, max_iter=10)

# Run MPC loop
y_ref = torch.tensor([0.23, 0.77], dtype=torch.float32)
z_current = torch.tensor([0.8, 0.2], dtype=torch.float32)

states = [z_current.numpy().copy()]
controls = []
for step in range(15):
    u_applied, info = mpc.step(z_current, y_ref)
    controls.append(u_applied.item())
    z_batch = z_current.unsqueeze(0)
    u_batch = u_applied.unsqueeze(0)
    with torch.no_grad():
        dzdt = model_ctrl.ode_func(torch.tensor(0.0), z_batch, u_batch).squeeze(0)
    z_current = z_current + dzdt * 0.2
    states.append(z_current.detach().numpy().copy())

states = np.array(states)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].plot(states[:, 0], 'b-o', markersize=3, label='C_A')
axes[0].plot(states[:, 1], 'r-o', markersize=3, label='C_B')
axes[0].axhline(y=y_ref[0].item(), color='b', linestyle='--', alpha=0.5, label='C_A ref')
axes[0].axhline(y=y_ref[1].item(), color='r', linestyle='--', alpha=0.5, label='C_B ref')
axes[0].set_xlabel('MPC Step'); axes[0].set_ylabel('Concentration')
axes[0].set_title('MPC Setpoint Tracking'); axes[0].legend(); axes[0].grid(True, alpha=0.3)

axes[1].step(range(len(controls)), controls, 'g-', where='post')
axes[1].set_xlabel('MPC Step'); axes[1].set_ylabel('Control Input')
axes[1].set_title('Control Actions'); axes[1].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Initial error: {np.sqrt(np.mean((states[0] - y_ref.numpy())**2)):.4f}")
print(f"Final error:   {np.sqrt(np.mean((states[-1] - y_ref.numpy())**2)):.4f}")

## 4. Online Adaptation

When the real plant drifts from the model's training distribution, online adaptation updates the Neural ODE parameters using new data. **Elastic Weight Consolidation (EWC)** prevents catastrophic forgetting by penalizing changes to parameters that were important for previously learned tasks.

In [ ]:
adapter = OnlineAdapter(
    model=model, buffer_size=100, ewc_lambda=100.0, lr=1e-4,
)

# Simulate new data arriving (slight distribution shift)
new_y0 = np.array([0.9, 0.1])
new_t = np.linspace(0, 5, 30)
new_sol = solve_ivp(reactor.ode_rhs, [0, 5], new_y0, t_eval=new_t, method="LSODA")

z0_new = torch.tensor(new_y0, dtype=torch.float32)
t_new = torch.tensor(new_t, dtype=torch.float32)
targets_new = torch.tensor(new_sol.y.T, dtype=torch.float32)

# Add experience and adapt
adapter.add_experience(z0_new, t_new, targets_new)
losses = adapter.adapt(num_steps=20)

print(f"Adaptation losses: {losses[0]:.6f} -> {losses[-1]:.6f}")
print(f"EWC prevents forgetting while learning new patterns")

# Consolidate for next round
adapter.consolidate()
print("Parameters consolidated for next EWC round")

## Summary

The digital twin layer provides a complete monitoring and control stack on top of the Neural ODE models:

| Component | Purpose | Key Feature |
|-----------|---------|-------------|
| **EKF State Estimator** | Fuse predictions with measurements | Optimal state estimates from noisy sensors |
| **SPC Charts** | Detect process anomalies | EWMA + CUSUM for different fault types |
| **MPC Controller** | Optimal control actions | Receding-horizon optimization using Neural ODE |
| **Online Adapter** | Continual learning | EWC prevents catastrophic forgetting |

Together, these components enable a Neural ODE to serve as a living digital twin that tracks the real plant, detects faults early, controls the process optimally, and adapts to changing conditions over time.